In [1]:
import findspark
findspark.init()
import pyspark
import random
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col
import pyspark.sql.functions
from pyspark.sql.functions import sum
from pyspark.sql.functions import countDistinct

In [2]:
spark = SparkSession.builder.master("local").appName("Yelp").getOrCreate()

In [3]:
sqlContext = SQLContext(spark)

In [4]:
business_df = sqlContext.read.json("yelp_academic_dataset_business.json")
checkin_df = sqlContext.read.json("yelp_academic_dataset_checkin.json")
review_df = sqlContext.read.json("yelp_academic_dataset_review.json")
tip_df = sqlContext.read.json("yelp_academic_dataset_tip.json")
user_df = sqlContext.read.json("yelp_academic_dataset_user.json")
photo_df = sqlContext.read.json("yelp_academic_dataset_photo.json")

In [9]:
review_df.select("business_id","user_id","date","stars").filter(col('business_id').isin(['OR6iRk0vrMzE-1gLg-WYrw'])).show(10,False)

+----------------------+----------------------+----------+-----+
|business_id           |user_id               |date      |stars|
+----------------------+----------------------+----------+-----+
|OR6iRk0vrMzE-1gLg-WYrw|YHWsLBS8jzZiPjKHMFOaAA|2010-11-22|2    |
|OR6iRk0vrMzE-1gLg-WYrw|EP_OQMoa4ZCSg-a7xNgrnA|2010-04-21|1    |
|OR6iRk0vrMzE-1gLg-WYrw|_YaW-qttstP5vlqsyaoMTw|2014-10-26|1    |
|OR6iRk0vrMzE-1gLg-WYrw|R7dm_9m_OrlMOZpfU2Dk9A|2013-05-22|2    |
|OR6iRk0vrMzE-1gLg-WYrw|pXwn9pMfDiuawK8airWsDA|2016-06-23|1    |
|OR6iRk0vrMzE-1gLg-WYrw|mmfZoI0npVJRDCnyVUE5tw|2017-03-18|4    |
|OR6iRk0vrMzE-1gLg-WYrw|zT9WFNJXeQZBA-WJMsV4Zw|2008-10-11|3    |
|OR6iRk0vrMzE-1gLg-WYrw|on4PBkuSAUi8gqlJHSyefA|2010-06-26|1    |
|OR6iRk0vrMzE-1gLg-WYrw|ZLY1CRMw8dWHWFm5VcRsXA|2010-06-04|1    |
|OR6iRk0vrMzE-1gLg-WYrw|SeLpdjMIdZz5KWOm1Q6Fag|2015-03-05|5    |
+----------------------+----------------------+----------+-----+
only showing top 10 rows



In [5]:
review_df.select("business_id","user_id","stars").filter(col('user_id').isin(['YHWsLBS8jzZiPjKHMFOaAA']))\
.filter(col('business_id').isin(['OR6iRk0vrMzE-1gLg-WYrw']))\
.show(10,False)

+----------------------+----------------------+-----+
|business_id           |user_id               |stars|
+----------------------+----------------------+-----+
|OR6iRk0vrMzE-1gLg-WYrw|YHWsLBS8jzZiPjKHMFOaAA|2    |
+----------------------+----------------------+-----+



In [6]:
business_df.select("business_id","city","categories").filter("city == 'Tempe'").show(5,False)

+----------------------+-----+------------------------------------------------------------+
|business_id           |city |categories                                                  |
+----------------------+-----+------------------------------------------------------------+
|XEGzpaKszjDXqSJtLYwKsg|Tempe|Beauty & Spas, Nail Salons                                  |
|od5dCR8QPU67t8Ku6CBezw|Tempe|Financial Services, Banks & Credit Unions                   |
|6nWyU5V3yINOq_DNWbpRDg|Tempe|Auto Parts & Supplies, Automotive                           |
|CcTvk2-unBwmAZmU3PlKww|Tempe|Restaurants, Delis, Sandwiches                              |
|WmY_00qgoiHDQydppWeJlQ|Tempe|Dentists, General Dentistry, Periodontists, Health & Medical|
+----------------------+-----+------------------------------------------------------------+
only showing top 5 rows



In [10]:
business_df2 = business_df.withColumn("review_count", business_df["review_count"].cast("double"))

In [11]:
business_df2.filter(col('city').isin(['Tempe'])).agg(sum("review_count")).show()

+-----------------+
|sum(review_count)|
+-----------------+
|         182638.0|
+-----------------+



In [12]:
business_review_count = business_df2.filter(col('city').isin(['Tempe'])).groupBy("business_id").\
agg(sum("review_count").alias("rev_count")).\
sort(col("rev_count").desc())

In [7]:
business_review_count.show(5)

+--------------------+---------+
|         business_id|rev_count|
+--------------------+---------+
|JzOp695tclcNCNMuB...|   2097.0|
|wl0QZqAzr1DelslQ0...|   1585.0|
|aiX_WP7NKPTdF9CfI...|   1291.0|
|5FIOXmUE3qMviX9Ga...|   1146.0|
|SurnOSM2bVVN4-Js3...|    963.0|
+--------------------+---------+
only showing top 5 rows



In [13]:
tempe_business_above5 = business_review_count.filter("rev_count >= 5")

In [14]:
tb5 = tempe_business_above5.alias('tb5')
rdf = review_df.alias('rdf')

In [15]:
tempe_bus_rev = rdf.join(tb5, rdf.business_id == tb5.business_id)\
.select('tb5.business_id','rdf.review_id','rdf.text','rdf.user_id','rdf.date','rdf.stars')

In [16]:
tempe_bus_rev.count()

179404

In [17]:
tempe_bus_rev.sort(col('business_id')).show(5)

+--------------------+--------------------+--------------------+--------------------+----------+-----+
|         business_id|           review_id|                text|             user_id|      date|stars|
+--------------------+--------------------+--------------------+--------------------+----------+-----+
|--9QQLMTbFzLJ_oT-...|vbDsxZPiLkIa8wWNI...|If you want a man...|zLtVHO6mfoRhx_6kZ...|2017-05-02|    5|
|--9QQLMTbFzLJ_oT-...|Hyrf_48taT5wcDa65...|Haircut was good ...|rJQe8TlW5_NYahQgf...|2016-11-19|    1|
|--9QQLMTbFzLJ_oT-...|D7OKYPWggteMDmvr6...|Always great cust...|NS2OQzrmJYHRXboib...|2013-08-18|    5|
|--9QQLMTbFzLJ_oT-...|2lbpUOGHRoISbRpzE...|Don't ever believ...|l6OM4eYMHVtjA3zK9...|2016-12-01|    1|
|--9QQLMTbFzLJ_oT-...|P_gU037tA8gWFDOkv...|I've been going t...|DdASELi5fpqRHm4ee...|2016-06-21|    5|
+--------------------+--------------------+--------------------+--------------------+----------+-----+
only showing top 5 rows



In [18]:
tempe_bus_rev.agg(countDistinct("business_id")).show()

+---------------------------+
|count(DISTINCT business_id)|
+---------------------------+
|                       3523|
+---------------------------+



In [27]:
tempe_bus_rev.agg(countDistinct("review_id")).show()

+-------------------------+
|count(DISTINCT review_id)|
+-------------------------+
|                   179404|
+-------------------------+



In [ ]:
tempe_bus_rev.select('business_id','user_id','stars').sort(col('business_id')).show(10,False)

In [33]:
testdf = tempe_bus_rev.select('').groupBy('user_id').count()

testRevdf = tempe_bus_rev.groupBy('business_id').count()


In [37]:
testRevdf.sort(col('count').desc()).show(5,False)

+----------------------+-----+
|business_id           |count|
+----------------------+-----+
|JzOp695tclcNCNMuBl7oxA|2096 |
|wl0QZqAzr1DelslQ02JGCQ|1585 |
|aiX_WP7NKPTdF9CfI-M-wg|1290 |
|5FIOXmUE3qMviX9GafGH-Q|1146 |
|SurnOSM2bVVN4-Js3G23RQ|963  |
+----------------------+-----+
only showing top 5 rows



In [38]:
testRevdf.filter(col('business_id').isin(['JzOp695tclcNCNMuBl7oxA'])).show()

+--------------------+-----+
|         business_id|count|
+--------------------+-----+
|JzOp695tclcNCNMuB...| 2096|
+--------------------+-----+



In [19]:
tempe_bus_rev.registerTempTable('tempe_bus_rev_table')

In [20]:
sqlContext.sql('select stars,count(*) from tempe_bus_rev_table group by stars').show()

+-----+--------+
|stars|count(1)|
+-----+--------+
|    5|   84486|
|    1|   27124|
|    3|   17403|
|    2|   14112|
|    4|   36279|
+-----+--------+



In [42]:
sqlContext.sql('select business_id, count(review_id) from tempe_bus_rev_table group by business_id sort by business_id').show()

+--------------------+----------------+
|         business_id|count(review_id)|
+--------------------+----------------+
|5GtfNFhdP-ws1BEDW...|               5|
|68htH_xVthYKRxvpT...|              20|
|DEBqmgxv2yhJ93LqG...|             111|
|DX94cQnhBJjE4I7bT...|              22|
|E9QTQ4DOKo1UsGNmM...|              71|
|FFe01fgrpADFJQ8WP...|              32|
|IEOHWxBkQD2kgt4NY...|              20|
|J6WnhToRaJ2TtLbRp...|              74|
|Nb5RUaRghxBNH6U3u...|               6|
|QlQ7qG7Hxk_SseGnr...|               5|
|TIaaQKCaJaWw0Q9m6...|               7|
|WXRaVM-k22cvq9KPO...|              11|
|Xz26ZNV3z3x3Lb5Lz...|              27|
|Z8emBxgaDgUf3BV2q...|              25|
|cg0RIQpKs0GhmZ_JC...|               6|
|i3cNlekSZoAPwcP2D...|              16|
|jj7zJSRfS3lb49zQ9...|              52|
|n0SMxITaU-6XrB0lO...|               8|
|vN6cLw7KKw62vbwOV...|              25|
|4jm-et6RMcjOkmrC_...|             132|
+--------------------+----------------+
only showing top 20 rows



In [43]:
sqlContext.sql('select business_id, count(user_id) from tempe_bus_rev_table group by business_id sort by business_id').show()

+--------------------+--------------+
|         business_id|count(user_id)|
+--------------------+--------------+
|5GtfNFhdP-ws1BEDW...|             5|
|68htH_xVthYKRxvpT...|            20|
|DEBqmgxv2yhJ93LqG...|           111|
|DX94cQnhBJjE4I7bT...|            22|
|E9QTQ4DOKo1UsGNmM...|            71|
|FFe01fgrpADFJQ8WP...|            32|
|IEOHWxBkQD2kgt4NY...|            20|
|J6WnhToRaJ2TtLbRp...|            74|
|Nb5RUaRghxBNH6U3u...|             6|
|QlQ7qG7Hxk_SseGnr...|             5|
|TIaaQKCaJaWw0Q9m6...|             7|
|WXRaVM-k22cvq9KPO...|            11|
|Xz26ZNV3z3x3Lb5Lz...|            27|
|Z8emBxgaDgUf3BV2q...|            25|
|cg0RIQpKs0GhmZ_JC...|             6|
|i3cNlekSZoAPwcP2D...|            16|
|jj7zJSRfS3lb49zQ9...|            52|
|n0SMxITaU-6XrB0lO...|             8|
|vN6cLw7KKw62vbwOV...|            25|
|4jm-et6RMcjOkmrC_...|           132|
+--------------------+--------------+
only showing top 20 rows



In [16]:
fnl_tempe_bus_df = tempe_bus_rev.select('user_id','business_id','stars')

fnl_tempe_bus_df.sort(col('user_id').desc()).show(5)

+--------------------+--------------------+-----+
|             user_id|         business_id|stars|
+--------------------+--------------------+-----+
|zzsmdXHUFBYuKUtPb...|U6P56MjLrraN0JBNv...|    5|
|zzsmdXHUFBYuKUtPb...|iqm-ZDFRpVGYe0Ie-...|    5|
|zzjIDvHGrYt2KK68I...|1Z4_zSITNVQ_Bt027...|    1|
|zzczJoSY3newDXLCD...|Wnq5to0-jXyh0-_jR...|    5|
|zz_aM1PBx68vNhVXF...|VX2aKMQ_v0x0G1r2E...|    1|
+--------------------+--------------------+-----+
only showing top 5 rows



In [22]:
fnl_tempe_with_date_df = tempe_bus_rev.select('business_id','date','stars','user_id')

In [23]:
fnl_tempe_with_date_df.show(5,False)

+----------------------+----------+-----+----------------------+
|business_id           |date      |stars|user_id               |
+----------------------+----------+-----+----------------------+
|UBv8heCQR0RPnUQG0zkXIQ|2016-09-23|1    |NhOc64RsrTT1Dls50yYW8g|
|hdgYnadxg0GANhWOJabr2g|2014-08-23|5    |NhOc64RsrTT1Dls50yYW8g|
|VfX7rhtX03yNg56ISz8gVQ|2016-06-08|4    |0pf5VuzE4_1pwj5NJHG5TQ|
|TT4XW2WMG0PNyUIFBIINuA|2015-10-18|1    |wn5sc78bQn-FpuR2u0rcCQ|
|x2NuIlQgvvl3Wtq8ipZC6Q|2015-10-23|5    |jjSR3JpsYKIgMUSR5zHRsQ|
+----------------------+----------+-----+----------------------+
only showing top 5 rows



In [24]:
sqlContext.sql('select business_id,date,stars,user_id from tempe_bus_rev_table order by business_id').show()

+--------------------+----------+-----+--------------------+
|         business_id|      date|stars|             user_id|
+--------------------+----------+-----+--------------------+
|--9QQLMTbFzLJ_oT-...|2018-02-17|    5|kolI5UE92RpFiBf8Y...|
|--9QQLMTbFzLJ_oT-...|2017-02-05|    1|mfdb2wCYv7yiu0fyz...|
|--9QQLMTbFzLJ_oT-...|2013-08-18|    5|NS2OQzrmJYHRXboib...|
|--9QQLMTbFzLJ_oT-...|2016-04-02|    2|ZPspeNuGDw5PYcu0m...|
|--9QQLMTbFzLJ_oT-...|2014-06-18|    5|WP6N1I1SCOIUyuBiJ...|
|--9QQLMTbFzLJ_oT-...|2016-11-19|    1|rJQe8TlW5_NYahQgf...|
|--9QQLMTbFzLJ_oT-...|2016-12-01|    1|l6OM4eYMHVtjA3zK9...|
|--9QQLMTbFzLJ_oT-...|2017-05-02|    5|zLtVHO6mfoRhx_6kZ...|
|--9QQLMTbFzLJ_oT-...|2013-05-15|    4|5RLxzxLIY3ZXUUksr...|
|--9QQLMTbFzLJ_oT-...|2016-06-21|    5|DdASELi5fpqRHm4ee...|
|--9QQLMTbFzLJ_oT-...|2015-08-13|    5|US0-y2ZX4O3uJtVjf...|
|--ab39IjZR_xUf81W...|2014-03-19|    5|jKDTqRBKpp3tjjbJH...|
|--ab39IjZR_xUf81W...|2014-08-02|    5|QN3ddcVHz2_I2ela_...|
|--ab39IjZR_xUf81W...|20

In [17]:
tempe_bus_rev_train, tempe_bus_rev_test = fnl_tempe_bus_df.randomSplit([0.8,0.2])

In [18]:
tempe_bus_rev_train.count()




143316

In [19]:
tempe_bus_rev_test.coalesce(1).write.mode("overwrite").csv('fnl_tempe_test.csv')

In [51]:
tempe_bus_rev_test.count()

35823

In [20]:
tempe_bus_rev_train.coalesce(1).write.mode("overwrite").csv('fnl_tempe_train.csv')

In [25]:
fnl_tempe_with_date_df.coalesce(1).write.mode("overwrite").csv('fnl_tempe_with_date.csv')